In [1]:
import re
import time
import datetime

import nltk
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu

from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from dataclasses import dataclass
from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import stopwords

pd.set_option('display.max_colwidth', None)
sns.set(style="darkgrid")
%matplotlib inline

In [2]:
def get_page(p) -> list:
    """A function to retrieve data from a web page."""

    # Generating the url for the request
    url = f'https://www.interfax.ru/news/{p}'
    
    # Receiving a response from the server
    response = requests.get(url)

    # Convert the response to a BeautifulSoup object
    tree = BeautifulSoup(response.content, 'html.parser')

    
    # Find all the news for the day
    news = tree.find_all('div', {'data-id' : re.compile('\d+')})

    
    info = []

    # Going around every news item
    for post in news:
        time = post.find('span')
        if time:
            time = time.text
        else:
            time = "?"
    
        title = post.h3.get_text()
        if title:
            title = title
        else:
            title = "?"

        link = post.a['href']
        id = link.split('/')[-1]
        
        if "https://www.interfax.ru/" in link:
            urli = link
        else:
            urli = 'https://www.interfax.ru' + link

        # Going inside each news item we found
        try:
            response_inner = requests.get(urli)
        except:
            continue
        tree_inner = BeautifulSoup(response_inner.content, 'html.parser')


        # If this is news from fontanka.ru
        if "https://www.interfax.ru/" in urli:
            topic = tree_inner.find('aside', {'class' : 'textML'}).a.get_text()
            contents = ''
            content = tree_inner.find('article', {'itemprop' : 'articleBody'}).find_all('p')
            for x in content:
                contents = contents + x.get_text() + ' '
            content = contents.strip()
            content = re.sub('.* INTERFAX.RU - ', '', content)

        else:
            content = "?"
            topic = "?"

        row = {
            'id': id,
            'date': p,
            'title': title,
            'topic': topic,
            'url': urli,
            'time': time,
            'content': content
            }

        info.append(row)
    
    return  info

In [3]:
get_page('2024/02/02')

[{'id': '944357',
  'date': '2024/02/02',
  'title': 'Глава Минобороны Армении заявил об отсутствии любых территориальных притязаний',
  'topic': 'В мире',
  'url': 'https://www.interfax.ru/world/944357',
  'time': '23:04',
  'content': 'Армения не имеет территориальных притязаний в отношении какой-либо страны, заявил министр обороны республики Сурен Папикян. "Цель и миссия наших ВС - защита суверенитета и территориальной целостности Армении. Эта концепция заложена в основу нашей армии", - сказал Папикян в интервью Общественному телевидению Армении. В минувший четверг премьер-министр Армении Никол Пашинян в разговоре с Общественным радио заявил, что страна укрепляет армию для защиты территориальной целостности республики, а не для "восстановления исторической справедливости". "Наша армия должна быть укреплена для защиты территориальной целостности, независимости, суверенитета нашей страны. Территориальная целостность нашей страны - это идентичная Армянской ССР территория", - сказал гла

In [4]:
%%time
"""
Passing through links in format:
'https://www.interfax.ru/news/{p}'
"""

infa = []
for yy in tqdm(range(2019, 2018, -1)):
    for mm in tqdm(range(12, 0, -1)):
        for dd in tqdm(range(31, 0, -1)):
            p = f'{yy:04}/{mm:02}/{dd:02}'
            try:
                infa.extend(get_page(p))
                df = pd.DataFrame(infa)
                df.to_csv('news_interfax_2019.csv', index=False, mode='a')
                infa = []
                
            except Exception as e:
                print(f"Failed to get data for: {p}. Exception: {e}")
                pass

  0%|                                                                                           | 0/12 [00:00<?, ?it/s]

  0%|                                                                                           | 0/31 [00:00<?, ?it/s]

  3%|██▋                                                                                | 1/31 [00:09<04:48,  9.62s/it]

  6%|█████▎                                                                             | 2/31 [00:20<05:07, 10.59s/it]

 10%|████████                                                                           | 3/31 [00:28<04:21,  9.35s/it]

 13%|██████████▋                                                                        | 4/31 [00:37<04:04,  9.05s/it]

 16%|█████████████▍                                                                     | 5/31 [00:47<04:05,  9.43s/it]

 19%|████████████████                                                                   | 6/31 [00:56<03:54,  9.38s/it]

 23%|██████████████████▋        

 87%|███████████████████████████████████████████████████████████████████████▍          | 27/31 [03:59<00:40, 10.01s/it]

 90%|██████████████████████████████████████████████████████████████████████████        | 28/31 [04:08<00:29,  9.77s/it]

 94%|████████████████████████████████████████████████████████████████████████████▋     | 29/31 [04:18<00:19,  9.87s/it]

 97%|███████████████████████████████████████████████████████████████████████████████▎  | 30/31 [04:26<00:09,  9.43s/it]

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [04:33<00:00,  8.83s/it]

 33%|███████████████████████████▎                                                      | 4/12 [18:18<36:29, 273.74s/it]

  0%|                                                                                           | 0/31 [00:00<?, ?it/s]

  3%|██▋                                                                                | 1/31 [00:07<03:55,  7.84s/it]

  6%|█████▎                     

 74%|████████████████████████████████████████████████████████████▊                     | 23/31 [03:05<01:01,  7.71s/it]

 77%|███████████████████████████████████████████████████████████████▍                  | 24/31 [03:14<00:56,  8.02s/it]

 81%|██████████████████████████████████████████████████████████████████▏               | 25/31 [03:23<00:49,  8.18s/it]

 84%|████████████████████████████████████████████████████████████████████▊             | 26/31 [03:32<00:43,  8.68s/it]

 87%|███████████████████████████████████████████████████████████████████████▍          | 27/31 [03:41<00:34,  8.67s/it]

 90%|██████████████████████████████████████████████████████████████████████████        | 28/31 [03:47<00:23,  7.86s/it]

 94%|████████████████████████████████████████████████████████████████████████████▋     | 29/31 [03:55<00:15,  7.84s/it]

 97%|███████████████████████████████████████████████████████████████████████████████▎  | 30/31 [04:01<00:07,  7.47s/it]

100%|███████████████████████████

 61%|██████████████████████████████████████████████████▎                               | 19/31 [02:52<01:47,  8.97s/it]

 65%|████████████████████████████████████████████████████▉                             | 20/31 [03:00<01:37,  8.90s/it]

 68%|███████████████████████████████████████████████████████▌                          | 21/31 [03:09<01:29,  8.93s/it]

 71%|██████████████████████████████████████████████████████████▏                       | 22/31 [03:19<01:22,  9.13s/it]

 74%|████████████████████████████████████████████████████████████▊                     | 23/31 [03:28<01:13,  9.17s/it]

 77%|███████████████████████████████████████████████████████████████▍                  | 24/31 [03:39<01:06,  9.55s/it]

 81%|██████████████████████████████████████████████████████████████████▏               | 25/31 [03:47<00:55,  9.22s/it]

 84%|████████████████████████████████████████████████████████████████████▊             | 26/31 [03:56<00:45,  9.11s/it]

 87%|███████████████████████████

CPU times: total: 5min 6s
Wall time: 53min 11s


In [2]:
x_1 = pd.read_csv('news_interfax_2023-2021.csv')
x_2 = pd.read_csv('news_interfax_2021-2019.csv')
x_3 = pd.read_csv('news_interfax_2019.csv')
df_final = pd.concat([x_1, x_2, x_3], axis=0)

In [4]:
df_final[df_final['id']=='id']

,id,date,title,topic,url,time,content
65,id,date,title,topic,url,time,content
119,id,date,title,topic,url,time,content
187,id,date,title,topic,url,time,content
255,id,date,title,topic,url,time,content
323,id,date,title,topic,url,time,content
...,...,...,...,...,...,...,...
22584,id,date,title,topic,url,time,content
22640,id,date,title,topic,url,time,content
22707,id,date,title,topic,url,time,content
22776,id,date,title,topic,url,time,content


In [7]:
df_final = df_final[(df_final['topic']!='topic')&(df_final['topic']!='Digital')&(df_final['topic']!='Интервью')]

In [9]:
df_final = df_final.reset_index()

In [10]:
df_final.loc[df_final['topic'] == 'В мире', 'topic'] = 'world'
df_final.loc[df_final['topic'] == 'Экономика', 'topic'] = 'ekonomika'
df_final.loc[df_final['topic'] == 'Культура', 'topic'] = 'kultura'
df_final.loc[df_final['topic'] == 'Москва', 'topic'] = 'Moscow'
df_final.loc[df_final['topic'] == 'В России', 'topic'] = 'Russia'

In [13]:
df_final['date'] = pd.to_datetime(df_final['date'], format='%Y/%m/%d', errors='coerce')

In [14]:
df_final = df_final[~df_final['date'].isna()]

In [15]:
df_lenta = pd.read_csv('lenta_news_cut.csv')

In [16]:
df_lenta.shape, df_final.shape

((383087, 8), (112899, 8))

In [17]:
df_lenta['categories'].value_counts()

Россия             126077
Мир                 90295
Экономика           57187
Спорт               35053
Наука и техника     29867
Культура            22353
Путешествия         22255
Name: categories, dtype: int64

In [18]:
df_lenta.loc[df_lenta['categories'] == 'Мир', 'categories'] = 'world'
df_lenta.loc[df_lenta['categories'] == 'Экономика', 'categories'] = 'ekonomika'
df_lenta.loc[df_lenta['categories'] == 'Культура', 'categories'] = 'kultura'
df_lenta.loc[df_lenta['categories'] == 'Россия', 'categories'] = 'Russia'
df_lenta.loc[df_lenta['categories'] == 'Спорт', 'categories'] = 'sport'
df_lenta.loc[df_lenta['categories'] == 'Наука и техника', 'categories'] = 'science_technology'
df_lenta.loc[df_lenta['categories'] == 'Путешествия', 'categories'] = 'traveling'

In [19]:
df_lenta['source'] = 'lenta'
df_final['source'] = 'interfax'

C:\Users\Мирослав\AppData\Local\Temp\ipykernel_31936\1405433721.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['source'] = 'interfax'


In [21]:
df_lenta = df_lenta.rename(columns={'categories':'topic', 'publishedAt':'date'})
df_lenta['date'] = df_lenta['date'].apply(lambda x: x.split('T')[0])
df_lenta['date'] = pd.to_datetime(df_lenta['date'], format='%Y/%m/%d')

In [23]:
df_lenta = df_lenta[['date', 'source', 'url', 'title','content', 'topic']]
df_final = df_final[['date', 'source', 'url', 'title','content', 'topic']]
df_lenta_interfax = pd.concat([df_lenta, df_final], axis=0)

In [34]:
df_lenta_interfax['topic'].value_counts()

Russia                169748
world                 129589
ekonomika              77595
sport                  35053
science_technology     29867
kultura                24028
traveling              22255
Name: topic, dtype: int64

In [33]:
df_lenta_interfax = df_lenta_interfax[df_lenta_interfax['topic']!='Moscow']

In [35]:
df_lenta_interfax.to_csv('df_lenta_interfax.csv')

In [36]:
df_lenta_interfax['source'].value_counts()

lenta       383087
interfax    105048
Name: source, dtype: int64